# Setup

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

In [ ]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Set the API keys as environment variables
# os.environ['OPENAI_API_KEY'] = openai_api_key

In [6]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0.5)

# Get the data

*   Interviewer
*   company
*   job position
*   job description



In [7]:
# Store the data
interviewer = input("Enter the name of the interviewer: ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer: Matthias Glöel
Enter the name of the company: Beat81
Enter the job position: Data Analytics Working Student
Enter the job description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science, data science, business in

# Interviewer AI Agent

In [11]:
# Create the interver AI agent
interviewer_agent = Agent(
    role = f" You are {interviewer}, employed at {company}, and hiring for job position {job_position}",
    goal = f" you help the user user prepare for the job interview for job {job_position} with the following description: {job_description}",
    backstory = """
    Experience
    BEAT81
    Analytics Engineer

    BEAT81 · Full-timeBEAT81 · Full-time
    Apr 2024 - Present · 11 mos
    Berlin, Germany
    diconium
    Full-time · 4 yrs 2 mosFull-time · 4 yrs 2 mos

    Senior Cloud Architect
    Apr 2022 - Mar 2024 · 2 yrs
    Microsoft Azure, DBT and +4 skills

    Data Engineer
    Jan 2021 - Mar 2022 · 1 yr 3 mosJan 2021 to Mar 2022 · 1 yr 3 mos
    DBT, AWS and +4 skills

    Data Scientist
    Feb 2020 - Dec 2020 · 11 mos
    SQL and Python
    4flow logo

    Data Scientist
    4flow · Full-time4flow · Full-time
    Jun 2019 - Dec 2019 · 7 mosJun 2019 to Dec 2019 · 7 mos
    SQL and Python
    DCMN
    Full-time · 3 yrs 3 mos

    Data Scientist
    Apr 2017 - Apr 2019 · 2 yrs 1 moApr 2017 to Apr 2019 · 2 yrs 1 mo
    SQL and Python
    Data Analyst / Business Intelligence Manager

    Feb 2016 - Mar 2017 · 1 yr 2 mosFeb 2016 to Mar 2017 · 1 yr 2 mos
    SQL and Python
    iversity logo

    Data Analyst
    iversity · Full-timeiversity · Full-time
    Dec 2014 - Dec 2015 · 1 yr 1 moDec 2014 to Dec 2015 · 1 yr 1 mo
    SQL and R

    Education
    Technische Universität Berlin
    Master of Science - MS, Computational Neuroscience
    Sep 2009 - Aug 2013

    Universität Osnabrück
    Bachelor of Science - BS, Cognitive ScienceBachelor of Science - BS, Cognitive Science
    """,
    llm = llm
)

In [35]:
# Define the interview prep task
interview_prep_task = Task(
    description = f"""
    Interview the user for the job {job_position} with the following description: {job_description}
    """,
    expected_output = f"""
    ask only one question to the user that is relevant for the job {job_position} that has not been asked before
    """,
    agent = interviewer_agent,
    human_input = True
)

# AI Coach

In [18]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position} by grading the relevance of the answer",
    backstory = "You are an expert in job interviews",
    llm = llm)

In [19]:
# Define the Coaching Task
coaching_task = Task(
    description = """ Give feedback to the user on its answer """,
    expected_output = f""" Give feedback to the user on its answer with a bullet points on what was good, bad, and how to take it to the next level""",
    agent = coach_agent,
    context = [interview_prep_task]
)

# AI Crew

In [32]:
# Create an empty list for questions
interview_questions = []
# # Store each question
# interview_questions.append(result.tasks_output[0].raw)
# interview_content = ' '.join(interview_questions)
# string_source = StringKnowledgeSource(content = interview_content)

In [36]:
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, coaching_task],
    verbose = True,
    process = Process.sequential,
    memory = True,
    knowledge_sources = [string_source]
)
result = crew.kickoff()

# Store each question
interview_questions.append(result.tasks_output[0].raw)
interview_content = ' '.join(interview_questions)
string_source = StringKnowledgeSource(content = interview_content)

# Agent:  You are Matthias Glöel, employed at Beat81, and hiring for job position Data Analytics Working Student
## Task: 
    Interview the user for the job Data Analytics Working Student with the following description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathe

In [28]:
# Check output
display(Markdown(result.tasks_output[0].raw))

What experience do you have working with SQL and Python, and how have you used these skills to provide analytical insights or solutions in past projects or coursework?